# Cvičení 02 - Rozhodovací stromy
  
  * V tomto notebooku byste se měli naučit, jak vytvořit (naučit) model dat ve formě rozhodovacího stromu.
  * Ukážeme si typický způsob, jak se toto dělá a představíme si i základní API knihovny `sklearn`.
  * Začneme s daty a skončíme s naučeným modelem, o kterém budeme tušit, jak je dobrý.

In [ ]:
import math
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline 

## Data: základní přehled

  * Použijeme balíček `pandas` k nahrání dat a jejich základnímu prozkoumání.
  * Data jsme stáhli ze serveru **kaggle.com**, kde můžete také najít [popis jednotlivých příznaků](https://www.kaggle.com/uciml/adult-census-income).
  * Cílem úlohy je na základě demografických údajů predikovat, zda příjem člověka přesáhne $50 000 ročně.

In [ ]:
data = pd.read_csv('adult-census.csv')
display(data.shape)
display(data.head())
# display(data.info())
display(data.describe())
display(data.nunique()) # počet unikátních hodnot, defaultně ignoruje NaNs

## Příprava dat před použitím modelu

Zkuste si později sami, že když vezmete aktuální data a pokusíte se je vložit do modelu `DecisionTreeClassifier`, který záhy budeme používat, tak to neprojde. 

Implementace rozhodovacího stromu v `sklearn` si neumí poradit s nečíselnými (nenumerickými) příznaky.
  * Musíme proto nahradit stringy nějakými jejich kódy.
  * **ÚKOL:** Zjistěte, jaký je vztah mezi příznaky *education* a *education.num*. 

In [ ]:
### sem přijde Váš kód


Vidíme, že `education.num` je vlastně číselná reprezentace té kategorie `education`.
Jedná se tedy o redundantní informaci, kterou v za chvíli odstraníme.

### Chybějící hodnoty
  * Některé příznaky jsou očividně nevyplněné, ale `pandas` hlásí, že žádné nechybí. Je to tím, že chybějící hodnoty nejsou zakódované jako `NaN` ale jsou nahrazeny otazníkem `?`.

In [ ]:
data = data.replace('?', np.nan)
# podívejme se na výsledek
data.info()
data.isnull().sum(axis=0)

### Převod textových dat na číselná data

Nejprve se podívejme, jaké jsou nečíselné příznaky.

In [ ]:
display(data.dtypes)
display(data.nunique())
string_cols = data.select_dtypes(['object']).columns
print(string_cols)
data[string_cols].nunique()

  * Všechny příznaky s hodnotami typu string nejsou spojité, ale tzv. **kategorické** (diskrétní).
  * Můžeme je tedy všechny nahradit číselnými kódy.
  * Je v `pandas` nějaká metoda, která to umí? Samozřejmě ano. Je ale nutné typ příznaku v `pandas` změnit na `category`.
  
Pro příznaky `income` a `education` definujeme kategorie ručně, abychom mohli zavést i uspořádání a ty číselné reprezentace aby to uspořádání respektovali (jedná se o takzvaná ordinální data).

U ostatních příznaků to necháme pandas udělat automaticky.

In [ ]:
data.income.unique()

In [ ]:
income_category = pd.api.types.CategoricalDtype(categories=['<=50K', '>50K'], ordered=True)
data['income'] = data['income'].astype(income_category)

In [ ]:
string_cols = data.select_dtypes(['object']).columns
print(string_cols)

Pro `education` využijeme již existující analýzy a z ní získaného uspořádání.

**Úkol:** Proveďte analogickou transformaci jako u `income`, ale využijte seřazení v `edu.sort_values(by="education.num")`

In [ ]:
display(edu.sort_values(by="education.num"))

In [ ]:
### sem přijde Váš kód

Ještě je převedeme na číselnou reprezentaci pomocí `...cat.codes`

In [ ]:
data.income = data.income.cat.codes
data.education = data.education.cat.codes
data.head()

In [ ]:
data.drop('education.num', axis = 1, inplace = True)

In [ ]:
data[string_cols] = data[string_cols].astype('category').apply(lambda x: x.cat.codes)
data.info()
# funguje to?
data["native.country"].unique()

### Chybějící hodnoty

* Co se stalo s `NaN` položkami?

**Chybějící hodnoty byly nahrazeny -1!**

Jak se vypořádat s chybějícími hodnotami je docela věda. Jejich nahrazení jedinou unikátní hodnotou je takový nejjednodušší (a skutečně používaný) způsob.

In [ ]:
data = data.replace('?', np.nan)
# podívejme se na výsledek
# data.info()
data.isnull().sum(axis=0)

## Příprava trénovací a validační množiny pro učení stromu

Nejprve si dataset rozdělíme na cílovou proměnnou, kterou chceme predikovat, a na matici příznaků, ze které budeme predikovat.

V našem případě je cílovou proměnnou **income**.

In [ ]:
Xdata = data.drop("income", axis = 1)
ydata = data.income

Dále si data rozdělíme na trénovací, validační  a testovací část.
- **Trénovací množinu** uvidí model při trénování. 
- **Validační množinu** používáme pro vyhodnocení výkonnosti modelu na datech, na kterých se netrénoval. 

Na základě validačních výsledků upravujeme hyperparametry modelu. Validační chyba je tedy zaujatá (až moc optimistická, vždyť jsme podle její hodnoty vybírali hyperparametry modelu!). 

Před odevzdáním finálního modelu do produkce obvykle chceme modelu dát co nejvyšší množství informací. Na poslední natrénování před testovací evaluací se obecně trénovací a validační množina sloučí, aby měl model nejvyšší možné množství dat.

- **Testovací dataset**: slouží pouze k evaluaci finálního (tedy již vybraného a natrénovaného modelu). 

Typicky ho má u sebe klient aby si ověřil, jak dobře dodané řešení funguje na datech, která váš model (ani váš data-science tým 👨‍🔬) neměl možnost při trénování vidět.

My si ho nyní také vytvoříme.

<div>
<img src="img/traintest.png" width="600"/>
</div>

K rozdělení dat použijeme [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) ze `sklearn`.

Tato funkce nám umožní rozdělit data (pseudo-náhodně) na dvě části. Pomocí opakovaného použití tak můžeme postupně vytvořit **trénovací**, **validační** a **testovací** část dat.

Data rozdělíme tak, aby 60% bylo trénovací, 20% validační a 20% testovací.

In [ ]:
rd_seed = 333 # data jsou rozdělována náhodně, v rámci testování je ale dobré mít pevně daný random seed
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(Xdata, ydata, test_size=0.4, random_state=rd_seed) 
display(Xtrain.shape)
display(ytrain.shape)
display(Xtest.shape)
display(ytest.shape)

In [ ]:
Xval, Xtest, yval, ytest = train_test_split(Xtest, ytest, test_size=0.5, random_state=rd_seed) 
display(Xval.shape)
display(yval.shape)
display(Xtest.shape)
display(ytest.shape)

## Vlastní částečná implementace stromu

Vytvoříme si pouze to prvotní dělení a nebude řešit otázku predikce. Pouze, abychom posléze viděli, že využití modelu z knihovny `scikit-learn` počítá to samé.

Jako kritérium při optimalizaci použijeme Gini index definovaý v pro dvě třídy přednášce jako $$GI = p_0(1-p_0) + p_1(1 - p_1),$$
kde $p_i$ je odhad pravděpodobnosti výskytu třídy $i$. V našem případě je $p_1$ počet $1$ dělený počtem všech a $p_0$ počet $0$ dělený počtem všech.  

Nejprve si naimplementujme funkci, které když dáme nějaký vektor (`series`) hodnot $y$, tak vrátí odpovídající Gini index.

In [ ]:
def GI(y):
    ...
    return ...

Dále si připravme počáteční hodnoty a pomocnou strukturu na ukládání aktuálního nejlepšího dělení.

In [ ]:
initGI = GI(ytrain)
print(f"Initial Gini {initGI}")
n = ytrain.shape[0]
print(f"N: {n}")

optimal = {'IG': 0,
          'col': None,
          'thr_val': None,
          'GI': 0,
          'GI1': 0,
          'GI2': 0,
          'N1': 0,
          'N2': 0}

A nyní už můžeme iterovat přes příznaky a přes všechny možné thresholdy a hledat optimální

In [ ]:
# iterate columns
for col in Xtrain.columns:
    print(f'Processing column {col}')
    # najdeme threshold
    values = sorted(Xtrain[col].unique())
    thrs = [(values[i-1] + values[i])/2 for i in range(1,len(values))]
    for thr_val in thrs:
        ...

Zobrazme si výsledek

In [ ]:
print(f"Dělení podle sloupce: {optimal['col']} s thresholdem {optimal['thr_val']}")
print(f"Originální Gini index: {optimal['GI']}\n")
print(f"Optimální IG: {optimal['IG']}")
print(f"Počty bodů po rozdělení: {optimal['N1']} a {optimal['N2']}")
print(f"Gini indexy po rozdělení: {optimal['GI1']} a {optimal['GI2']}")

## Použití rozhodovacího stromu implementovaného v `sklearn`
 
 * [Zde](http://scikit-learn.org/stable/modules/tree.html) najdete dokumentaci k rozhodovacím stromům v balíčku `tree` knihovny `scikit-learn`.


In [ ]:
from sklearn.tree import DecisionTreeClassifier

* V knihovně `scikit-learn` mají modely jednotný interface (API).
* Nejdříve se vždy nastaví hyperparametry modelu (jaké jsou k dispozici lze zjistit pomocí nápovědy) a "nastavený" model se uloží do proměnné:

In [ ]:
clf = DecisionTreeClassifier(max_depth = 1)
clf.get_params()

* Model se pak "naučí" na datech pomocí metody `.fit()`. 
* V případě supervizovaných modelů vyžaduje většinou matici (`pandas.DataFrame`) s příznaky a vektor (`pandas.Series`) s vysvětlovanou proměnnou.

In [ ]:
clf.fit(Xtrain, ytrain)

## Porovnání s naší vlastní implementací

Použijeme k tomu `plot_tree` funkci ze `sklearn`.

In [ ]:
from sklearn.tree import plot_tree

plt.figure(figsize=(10,10))
plot_tree(clf, filled=True, feature_names=Xtrain.iloc[:,:].columns, class_names=True, rounded=True)

**Vidíme, že dostáváme to samé!**

V dalším už budeme pracovat pouze s implementací ze `sklearn`.

## Měření trénovací chyby pomocí klasifikační přesnosti

In [ ]:
clf = DecisionTreeClassifier(max_depth=50)
clf.fit(Xtrain, ytrain)

  * Rozhodovací strom máme hotový, nyní můžeme použít metodu `predict()`, abychom získali predikce, které dává strom pro trénovací data.

In [ ]:
ypredicted = clf.predict(Xtrain)
ypredicted

In [ ]:
(ytrain - ypredicted).value_counts(normalize=True)

  * Přesnost na trénovacích datech je  (pro `max_depth` = 50) skoro 100 %! 
  * Měřit přesnost a různé metriky nemusíme takto ručně, v `sklearn` je na to samozřejmě balíček:

In [ ]:
import sklearn.metrics as metrics
print('accuracy score (train): {0:.6f}'.format(metrics.accuracy_score(ytrain, clf.predict(Xtrain))))

S testovacími daty to tak skvěle nedopadne:

In [ ]:
print('accuracy score (test): {0:.6f}'.format(metrics.accuracy_score(ytest, clf.predict(Xtest))))

### Ladění hyperparameterů `max_depth` a `criterion`

  * Začněme s hyperparametrem `max_depth`.
  * Jak víme, budeme používat validační množinu.
  * Prozkoumáme hodnoty od 1 do 100.

In [ ]:
val_acc = []
train_acc = []
depths = range(1,101)
for depth in depths:
    clf = DecisionTreeClassifier(max_depth=depth)
    clf.fit(Xtrain, ytrain)
    train_acc.append(metrics.accuracy_score(ytrain, clf.predict(Xtrain)))
    val_acc.append(metrics.accuracy_score(yval, clf.predict(Xval)))

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
plt.figure(figsize=(20,6))
plt.plot(depths, train_acc,'or-') #o: body, r: barva, -: spojení bodů linkou
plt.plot(depths, val_acc,'ob-')
plt.xlabel('max_depth')
plt.ylabel('accuracy')
plt.legend(['train', 'validation'])

In [ ]:
optimal_depth = np.argmax(val_acc) + 1
optimal_depth

Spočítejme **testovací accuracy** pro nejlepší nalezenou hodnotu `max_depth`:

In [ ]:
clf = DecisionTreeClassifier(max_depth=optimal_depth)
clf.fit(Xtrain, ytrain)
print('accuracy score (test): {0:.6f}'.format(metrics.accuracy_score(ytest, clf.predict(Xtest))))

### Grid search: ladění více hyperparameterů

  * Chceme projít hodnoty `max_depth` z intervalu [1, 100] a obě možné hodnoty `criterion`, tj. 200 různých kombinací.
  * Můžeme samozřejmě použít dva vnořené for cykly (což je hrozné) nebo použít `ParameterGrid` balíček.

In [ ]:
from sklearn.model_selection import ParameterGrid
param_grid = {
    'max_depth': range(1,30), 
    'criterion': ['entropy', 'gini']
}
param_comb = ParameterGrid(param_grid)
# just to see the result:
[param_comb[i] for i in range(2)]

In [ ]:
val_acc = []
train_acc = []
for params in param_comb:
    clf = DecisionTreeClassifier(max_depth=params['max_depth'], criterion=params['criterion'])
    clf.fit(Xtrain, ytrain)
    train_acc.append(metrics.accuracy_score(ytrain, clf.predict(Xtrain)))
    val_acc.append(metrics.accuracy_score(yval, clf.predict(Xval)))

In [ ]:
plt.figure(figsize=(20,6))
plt.plot(train_acc,'or-')
plt.plot(val_acc,'ob-')
plt.xlabel('hyperparametr index')
plt.ylabel('accuracy')
plt.legend(['train', 'validation'])

In [ ]:
best_params = param_comb[np.argmax(val_acc)]
best_params

In [ ]:
clf = DecisionTreeClassifier(**best_params) # TOHOTO ŘÁDKU si všimněte: dvě hvězdišky jsou mocný nástroj
clf.fit(Xtrain, ytrain)
print('accuracy score (train): {0:.6f}'.format(metrics.accuracy_score(ytrain, clf.predict(Xtrain))))
print('accuracy score (validation): {0:.6f}'.format(metrics.accuracy_score(yval, clf.predict(Xval))))
print('accuracy score (test): {0:.6f}'.format(metrics.accuracy_score(ytest, clf.predict(Xtest))))

### ÚKOL: naučte strom na datech k Titaniku z minulého cvičení

  * Hyperparametry k tunění si vyberte sami.
  * Rozdělte data ve stejných poměrech a se stejným `random_state` v metode `train_test_split()` jeko výše.

In [ ]:
titanic = pd.read_csv('titanic.csv', sep=';')
titanic.nunique()
#  #unikátních hodnot = #řádků
titanic.drop(columns=["Name", "PassengerId"], inplace=True)

# kategorie
string_cols = titanic.select_dtypes(['object']).columns
titanic[string_cols] = titanic[string_cols].astype('category').apply(lambda x: x.cat.codes)
display(titanic.head())

# nahrazení nans
titanic = titanic.replace(np.nan, -1)
display(titanic.head())

# rozsekání dat
Xdata = titanic.iloc[:,1:]
ydata = titanic.iloc[:,0]

############################################################
############################################################
### sem přijde Váš kód


